# Descrição

Este projeto busca modelar em um grafo as relações entre empresa - orgão publico, quando ocorre uma licitação. Assim, podemos pensar que o grafo poderá ser dividido em dois subconjuntos de vértices (empresa e orgão público), ou seja, será um grafo bipartido.


## Dados

-   Os dados foram obtidos através do portal da transparência, https://portaldatransparencia.gov.br/download-de-dados, na aba de licitações e contratos.
-   Para limitar o escopo da análise, foi definido que apenas licitações do ano de 2022 serão utilizadas. E além disso, as analises serão feitas apenas em licitações que ocorreram na município Curitiba.
-   A pasta baixada possui as seguintes tabelas:
    -   Licitacoes: informações sobre a licitação.
    -   Participantes: informações sobre as empresas participantes da licitação. 
    -   Itens: informações sobre os itens soliticados.
    -   Empenhos Relacionados: informações sobre o empenho que é a etapa em que o governo reserva o dinheiro que será pago quando o bem for entregue ou o serviço concluído.
-   As pastas foram baixadas mês a mês.
-   As tabelas foram concatenadas e depois unidas para gerar um dataset final.
-   O dataset final ficou composto das seguintes informações:
    -   Número Licitação
    -   Código UG
    -   Nome UG
    -   Código Modalidade Compra
    -   Modalidade Compra
    -   Número Processo
    -   Objeto
    -   Situação Licitação
    -   Código Órgão Superior
    -   Nome Órgão Superior
    -   Código Órgão
    -   Nome Órgão
    -   UF
    -   Municipio
    -   Data Resultado Compra
    -   Data Abertura
    -   Valor Licitação
    -   Código Item Compra
    -   Descrição
    -   Quantidade Item
    -   Valor do Item
    -   Código Vencedor
    -   Nome Vencedor

Projeção

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

import networkx as nx

%matplotlib inline

### Load Data

In [73]:
def concatena_csv_em_pasta(pasta, columns_to_read=None):
    # Verifica se a pasta existe
    if not os.path.exists(pasta):
        return None

    arquivos_csv = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.csv')]

    if not arquivos_csv:
        return None

    # Cria uma lista para armazenar os DataFrames dos arquivos CSV
    dfs = []

    for arquivo_csv in arquivos_csv:
        caminho_arquivo = os.path.join(pasta, arquivo_csv)
        df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', usecols=columns_to_read)
        print(df)
        dfs.append(df)

    # Concatena todos os DataFrames em um único DataFrame
    dataframe_concatenado = pd.concat(dfs, ignore_index=True)

    return dataframe_concatenado

licitacoes = '../data_APS/licitacoes_2022'
participantes = '../data_APS/participantes_2022'
itens = '../data_APS/itens_2022'
empenhos_relacionados = '../data_APS/empenhosRelacionados_2022'


df_licitacoes22 = concatena_csv_em_pasta(licitacoes)
df_participantes22 = concatena_csv_em_pasta(participantes,  ['Número Licitação','Código UG', 'Código Participante', 'Nome Participante', 'Flag Vencedor'])
df_itens22 = concatena_csv_em_pasta(itens,['Número Licitação','Código UG', 'Código Item Compra', 'Descrição', 'Quantidade Item', 'Valor Item', 'Código Vencedor', 'Nome Vencedor'])
df_empenhos_relacionados22 = concatena_csv_em_pasta(empenhos_relacionados, ['Número Licitação','Código UG', 'Código Empenho', 'Observação Empenho', 'Valor Empenho (R$)'])

      Número Licitação  Código UG  \
0                12020     413006   
1                12021     120629   
2                12021     154849   
3                12021     158308   
4                12021     158373   
...                ...        ...   
5955         901032021     926137   
5956         901062021     926137   
5957         910572021     120060   
5958         911522021     120060   
5959         988512022     154040   

                                           Nome UG  Código Modalidade Compra  \
0          AGENCIA NACIONAL DE TELECOMUNICACOES-RS                         6   
1                    GRUPAMENTO DE APOIO DE CANOAS                         2   
2            IFPE - CAMPUS CABO DE SANTO AGOSTINHO                       -99   
3                INST.FED.DO PARA/CAMPUS CASTANHAL                       -99   
4                INST.FED.DO RN/CAMPUS JOAO CAMARA                         5   
...                                            ...                       ..

/tmp/ipykernel_65363/3838231747.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', usecols=columns_to_read)


        Número Licitação  Código UG Código Participante  \
0                  12019     495600      10761735000191   
1                  12019     495600      20204978000182   
2                  12019     495600      30814518000120   
3                  12019     495600      20421441000174   
4                  12021     160059      19037262000159   
...                  ...        ...                 ...   
418304         938862022     910809      03750414000126   
418305         938862022     910809      03750414000126   
418306         938862022     910809      03750414000126   
418307         938862022     910809      03750414000126   
418308         967302022     154040      42595652000166   

                                        Nome Participante Flag Vencedor  
0                  BR DANTAI DISTRIBUIDOR HOSPITALAR LTDA           NÃO  
1                          MBR COMERCIO DE MATERIAIS LTDA           NÃO  
2       MANHUACU CONSTRUCAO, TERCEIRIZACAO E COMERCIAL...           N

/tmp/ipykernel_65363/3838231747.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', usecols=columns_to_read)


        Número Licitação  Código UG Código Participante  \
0                  12022     929546      33157312000162   
1                  12022     929546      19207352000140   
2                  12022     929546      69034668000156   
3                  12022     929546      12228728000154   
4                  12022     114605      27361863000140   
...                  ...        ...                 ...   
454395         993092022     910809      16925627000193   
454396         993092022     910809      04438673000189   
454397         993092022     910809      11223241000116   
454398         993092022     910809      14631732000194   
454399         994872022     154040      28019941000196   

                                        Nome Participante Flag Vencedor  
0                       IFOOD BENEFICIOS E SERVICOS LTDA.           NÃO  
1                  LE CARD ADMINISTRADORA DE CARTOES LTDA           NÃO  
2          SODEXO PASS DO BRASIL SERVICOS E COMERCIO S.A.           N

/tmp/ipykernel_65363/3838231747.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', usecols=columns_to_read)


        Número Licitação  Código UG Código Participante  \
0                  12021     170175      09006742000107   
1                  12022     925134      31907225000150   
2                  12022     925134      36895360000146   
3                  12022     925134      35389852000105   
4                  12022     925134      45225296000187   
...                  ...        ...                 ...   
500510         910732021     120060      07422196000151   
500511         910732021     120060      24262316000110   
500512         910732021     120060      07422196000151   
500513         910732021     120060      24262316000110   
500514         922022022     120625      37108388000159   

                                        Nome Participante Flag Vencedor  
0                                  ATELY CONSTRUTORA LTDA           SIM  
1       VEC COMERCIO DE INSTRUMENTOS E MATERIAIS PARA ...           NÃO  
2                              VALENCY TRADE SERVICE LTDA           N

/tmp/ipykernel_65363/3838231747.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', usecols=columns_to_read)


        Número Licitação  Código UG Código Participante  \
0                  12019     153080      40884553000179   
1                  12022     110001      39371850000131   
2                  12022     110161      33402892000106   
3                  12022     114615      45456117000112   
4                  12022     120060      08377693000148   
...                  ...        ...                 ...   
480514         910082022     120060      55956510000129   
480515         910082022     120060      27263741000111   
480516         910082022     120060      08825548000182   
480517         910082022     120060      17370546000137   
480518         920072022     153035      33683111000107   

                                        Nome Participante Flag Vencedor  
0            CONCREFERRO CONSTRUCOES E INCORPORACOES LTDA           SIM  
1                     ROMA EDIFICACOES E CONSTRUCOES LTDA           SIM  
2           ASSOCIACAO BRASILEIRA DE NORMAS TECNICAS ABNT           S

In [3]:
df_licitacoes22.head()

,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,UF,Município,Data Resultado Compra,Data Abertura,Valor Licitação
0,12020,413006,AGENCIA NACIONAL DE TELECOMUNICACOES-RS,6,Dispensa de Licitação,53528000739202091,Objeto: Aquisição de Materiais de Expediente p...,Encerrado,41000,Ministério das Comunicações,41231,Agêncial Nacional de Telecomunicações,RS,PORTO ALEGRE,14/02/2022,23/04/2020,"3035,8000"
1,12021,120629,GRUPAMENTO DE APOIO DE CANOAS,2,Tomada de Preços,67278003552202123,Objeto: Contratação de empresa de engenharia e...,Publicado,52000,Ministério da Defesa,52111,Comando da Aeronáutica,RS,CANOAS,10/02/2022,NaN,"779021,2300"
2,12021,154849,IFPE - CAMPUS CABO DE SANTO AGOSTINHO,-99,Pregão - Registro de Preço,23518004708202148,Objeto: Pregão Eletrônico - Registro de Preço...,Publicado,26000,Ministério da Educação,26418,Instituto Federal de Pernambuco,PE,CABO DE SANTO AGOSTINHO,15/02/2022,10/01/2022,"29550,0000"
3,12021,158308,INST.FED.DO PARA/CAMPUS CASTANHAL,-99,Pregão - Registro de Preço,23051001705202193,Objeto: Pregão Eletrônico - Aquisição de seme...,Publicado,26000,Ministério da Educação,26416,Instituto Federal do Pará,PA,CASTANHAL,14/02/2022,03/02/2022,"3243,0000"
4,12021,158373,INST.FED.DO RN/CAMPUS JOAO CAMARA,5,Pregão,23134002705202184,Objeto: Pregão Eletrônico - Recuperação emerg...,Evento de Resultado de Julgame,26000,Ministério da Educação,26435,Instituto Federal do Rio Grande do Norte,RN,JOAO CAMARA,09/02/2022,13/12/2021,"147185,9100"


In [74]:
df_licitacoes22.rename(columns={'Município': 'Municipio'}, inplace=True)
df_licitacoes22['Situação Licitação'].value_counts()

Situação Licitação
Encerrado                         62569
Publicado                         30831
Evento de Resultado de Julgame    10357
Evento de Suspensão Publicado      1236
Evento de Alteração Publicado       953
Divulgado                           720
Evento de Adiamento Publicado       480
Revogação                           434
Anulação                            418
Evento de Revogação Publicado       341
Evento de Alteração de Resulta      199
Evento de Anulação Publicado        195
Retificação                         159
Evento de Retificação Publicad       79
Evento de Retificação Divulgad       47
Evento de Adiamento Divulgado        45
Evento de Reabertura de Prazo        38
Evento de Alteração Divulgado        32
Inválido                             32
Evento de Revogação Divulgado        18
Evento de Habilitação Publicad        8
Pendente de Processamento             3
Evento de Alteração de Habilit        2
Name: count, dtype: int64

In [75]:
df_licitacoes_ctba = df_licitacoes22[df_licitacoes22.Municipio == 'CURITIBA']
df_licitacoes_ctba["code"] = df_licitacoes_ctba["Número Licitação"].astype(str) + "_" + df_licitacoes_ctba["Código UG"].astype(str)

/tmp/ipykernel_65363/3110833496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_licitacoes_ctba["code"] = df_licitacoes_ctba["Número Licitação"].astype(str) + "_" + df_licitacoes_ctba["Código UG"].astype(str)


In [76]:
map_orgao_by_code = df_licitacoes_ctba.set_index('code')['Código Órgão'].to_dict()

In [77]:
df_participantes22["code"] = df_participantes22["Número Licitação"].astype(str) + "_" + df_participantes22["Código UG"].astype(str)

In [78]:
df_participantes_ctba = df_participantes22[df_participantes22['code'].isin(df_licitacoes_ctba['code'])]

In [79]:
df_participantes_ctba.value_counts('Código Participante')

Código Participante
17323941000169    818
00802002000102    704
42262411000103    490
08658622000113    469
00096764000122    419
                 ... 
25333150000148      1
25358034000183      1
25371614000100      1
25897729000133      1
ESTRANG0033431      1
Name: count, Length: 6471, dtype: int64

In [116]:
edges_df = df_participantes_ctba[['Número Licitação', 'Código UG', 'Código Participante', 'Flag Vencedor', 'code']]
# for index, row in edges_df.iterrows():
#     edges_df.loc[index, 'Código Órgão'] = map_orgao_by_code[row['code']]



In [117]:
# drop code column
edges_df.drop(columns=['code'], inplace=True)

/tmp/ipykernel_65363/1516726114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_df.drop(columns=['code'], inplace=True)


In [ ]:
# df_participantes_ctba22 = df_participantes22_selec[df_participantes22_selec['Flag Vencedor'] == 'SIM']
# df_participantes_ctba22

In [93]:
# group codes by name

df_participantes_ctba22 = df_participantes_ctba.groupby(['Nome Participante']).agg({'Código Participante': lambda x: list(x)}).reset_index()

In [94]:
df_participantes_ctba22.__len__()

6171

In [105]:
map_codes_by_code = {}

for index, row in df_participantes_ctba22.iterrows():
    codes = row['Código Participante']
    for code in codes:
        map_codes_by_code[code] = codes[0]

diferentes = 0
iguais = 0
for key, value in map_codes_by_code.items():
    if key == value:
        iguais += 1
        continue
    diferentes += 1

print(f'Diferentes: {diferentes}')
print(f'Iguais: {iguais}')

raw_nodes = map_codes_by_code.values()

nodes = list(set(raw_nodes))


Diferentes: 300
Iguais: 6171


In [106]:
nodes.__len__()

6171

In [142]:
G = nx.Graph()
G.add_nodes_from(nodes, bipartite=0)
G.add_nodes_from(df_licitacoes_ctba['Código UG'], bipartite=1)


# color each side of bipartite graph
color_map = []
for node in G:
    if G.nodes[node]['bipartite'] == 0:
        nx.set_node_attributes(G, {node: {"Nome Participante": df_participantes_ctba[df_participantes_ctba['Código Participante'] == node]['Nome Participante'].values[0]}}) 
        color_map.append('blue')
    else:
        nx.set_node_attributes(G, {node: {"Nome UG": df_licitacoes_ctba[df_licitacoes_ctba['Código UG'] == node]['Nome UG'].values[0]}})
        color_map.append('green')


for index, row in edges_df.iterrows():
    # check if edge already exists
    if G.has_edge(map_codes_by_code[row['Código Participante']], row['Código UG']):
        G.edges[map_codes_by_code[row['Código Participante']], row['Código UG']]['qnt'] += 1
        if row['Flag Vencedor'] == 'SIM':
            G.edges[map_codes_by_code[row['Código Participante']], row['Código UG']]['success'] += 1
        qnt = G.edges[map_codes_by_code[row['Código Participante']], row['Código UG']]['qnt']
        success = G.edges[map_codes_by_code[row['Código Participante']], row['Código UG']]['success']
        G.edges[map_codes_by_code[row['Código Participante']], row['Código UG']]['success_rate'] = success/qnt
        continue


    qntWinner = 0
    if row['Flag Vencedor'] == 'SIM':
        qntWinner = 1

    G.add_edge(map_codes_by_code[row['Código Participante']], row['Código UG'], success=qntWinner, qnt=1, success_rate=qntWinner)


# calc node success rate

for node in G:
    if G.nodes[node]['bipartite'] == 1:
        continue
    success = 0
    qnt = 0
    for edge in G.edges(node, data=True):
        success += edge[2]['success']
        qnt += edge[2]['qnt']
    G.nodes[node]['success_rate'] = success/qnt
    G.nodes[node]['licitações'] = qnt

In [143]:
# save to edgelist

nx.write_gexf(G, "../data_APS/licitacoes_2022.gexf")


In [111]:
nx.is_connected(G)

False

In [ ]:
nx.betweenness_centrality(G)

{'ILUMINAR COMERCIO E REPRESENTACOES LTDA': 0.0,
 'MORK SOLAR - PRODUTOS E SERVICOS ELETRICOS LTDA': 0.002822433989645369,
 'COMERCIAL SPONCHIADO LTDA': 0.0012719380791937328,
 'J. J. VITALLI': 0.002750652511994846,
 'GIOVANI LOS': 0.0006951246898977856,
 'LICITARE PRODUTOS, MATERIAIS E SERVICOS LTDA': 0.00026472013694202296,
 'SZATA COMERCIO VAREJISTA LTDA': 0.002822433989645369,
 'MD COMERCIO E SERVICOS DE PINTURA LTDA': 0.0012719380791937328,
 'I.R. COMERCIO E MATERIAIS ELETRICOS LTDA': 0.00026472013694202296,
 'MULTI LITE COMERCIAL ELETRICA LTDA.': 0.0006951246898977856,
 'PROLUX ILUMINACAO LTDA': 0.0006555399294505092,
 'LIBERTY COMERCIO E ATACADISTA LTDA': 0.0,
 'TI COMPONENTES ELETRONICOS LTDA': 0.0,
 'LX DISTRIBUIDORA DE MATERIAIS ELETRICOS LTDA': 0.0012722419608205424,
 'EZ TECHS IMPORTADORA, EXPORTADORA E REPRESENTACOES LTDA': 0.0012722419608205424,
 'EMPALUX ELETROSHOP LTDA': 0.0,
 'GR COMERCIO LTDA': 0.0012722419608205424,
 'OLIVEIRA ARAUJO ENGENHARIA LTDA': 0.0,
 'ENGEPLAN

In [112]:

# remove non connected nodes

G.remove_nodes_from(list(nx.isolates(G)))


participant_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
bidding_nodes = set(G) - participant_nodes

# project bipartite graph onto participant nodes
P = nx.bipartite.projected_graph(G, participant_nodes)



In [113]:
# write projected graph to file

nx.write_gexf(P, "../data_APS/licitacoes_2022_projected_participantes.gexf")


In [134]:
out = nx.to_pandas_edgelist(G)

In [135]:
out.head()

,source,target,success_rate,qnt,success
0,16876380000162,153079,0.583333,12,7
1,34296901000194,160223,1.000000,1,1
2,09649306000148,153079,0.000000,4,0
3,21366890000120,153079,0.000000,6,0
4,21366890000120,158009,0.000000,1,0


In [136]:
sort_by = 'success_rate'

out.sort_values(by=sort_by, ascending=False, inplace=True)

out.head(10)

,source,target,success_rate,qnt,success
10873,84945401000104,130070,1.0,2,2
5409,53617676000195,158009,1.0,1,1
5467,10960298000135,158009,1.0,7,7
5463,05579646000199,160223,1.0,1,1
5460,06085483000150,160220,1.0,1,1
5459,06085483000150,158009,1.0,1,1
1633,39649812000106,160223,1.0,1,1
5454,22355622000175,155902,1.0,2,2
5449,44442386000167,160211,1.0,1,1
5440,11375069000116,153079,1.0,17,17
